In [49]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF
from arcgis.features import GeoAccessor, GeoSeriesAccessor, Feature

from arcgis.gis import GIS, Item

import sys
sys.path.append('..')

import geometry_monkeypatch
import reach_tools

In [2]:
reach_item_id = 'c72a6e88a67e4eafb7b291f17ec0cf18'
reach_layer_url = 'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/Reach_Line/FeatureServer/0'
reach_id_ldub = 2156
reach_id_farmies = 2269
reach_id_canyon = 3066

In [126]:
import importlib
importlib.reload(reach_tools)

reach = reach_tools.Reach.get_from_aw(reach_id_ldub)

reach.update_putin_takeout_and_trace()

reach.geometry

{'paths': [[(-121.633094507621, 45.7953236766543),
   (-121.632808521595, 45.7949415287295),
   (-121.632717521896, 45.7943735286163),
   (-121.6334955218, 45.7932515290357),
   (-121.633564521385, 45.7930455285292),
   (-121.633434521685, 45.7927295292453),
   (-121.632061521322, 45.7921535287633),
   (-121.631961388107, 45.7919015288334),
   (-121.632457388497, 45.7912375287898),
   (-121.63229038799, 45.7909205285604),
   (-121.631244388317, 45.7902535292779),
   (-121.631282388271, 45.7898645293255),
   (-121.632885388449, 45.7883655285472),
   (-121.633014321553, 45.7874455292887),
   (-121.632686321716, 45.7869235285989),
   (-121.631969321926, 45.7865345286465),
   (-121.629261188053, 45.7854585959384),
   (-121.628185187896, 45.7846535956878),
   (-121.627140188269, 45.7835315952079),
   (-121.626941121535, 45.7830555957401),
   (-121.627071121235, 45.7822085953513),
   (-121.628055121646, 45.7815485954922),
   (-121.628444121598, 45.7814335955849),
   (-121.628711121321, 45.78

In [132]:
reach.centroid

(-121.63420429856335, 45.75496489434612)

In [131]:
web_map = GIS().map()

In [10]:
web_map.draw(reach.putin.geometry)
web_map.draw(reach.takeout.geometry)

In [180]:
resp_json = waters._get_point_indexing(reach.putin.geometry.x, reach.putin.geometry.y)

In [181]:
resp_json['output'] == None

True

In [194]:
importlib.reload(reach_tools)
reach = reach_tools.Reach.get_from_aw(reach_id_ldub)

In [195]:
import numpy as np
np.mean([reach.putin.geometry.x, reach.takeout.geometry.y])

-37.9577925

In [196]:
reach.centroid

{'x': -121.639992, 'y': 45.7568325, 'spatialReference': {'wkid': 4326}}

In [197]:
web_map.draw(reach.centroid)

In [133]:
gis = GIS(url='https://jdm1cc.maps.arcgis.com', username='joel_mccune')
reach_layer = reach_tools.ReachFeatureLayer(reach_layer_url, gis)

In [134]:
reach_layer.flush()

In [135]:
for reach_id in range(0, 10000):
    print('reach_id -', reach_id)
    reach = reach_tools.Reach.get_from_aw(reach_id)
    if type(reach) == reach_tools.Reach:
        trace_status = reach.update_putin_takeout_and_trace()
        if trace_status == True:
            resp = reach.publish(gis, reach_layer)
            print(resp)
        else:
            print('/no trace')
    else:
        print('\tno reach')

reach_id - 0
	no reach
reach_id - 1


TypeError: 'bool' object is not subscriptable